In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split


from tensorflow.keras.preprocessing.image import ImageDataGenerator

import cv2


In [2]:
print(os.listdir('own_dataset/1d20/train'))

['03', '12', '14', '07', '13', '20', '17', '04', '08', '05', '15', '01', '18', '11', '16', '10', '09', '06', '19', '02']


In [3]:
train_path= ('own_dataset/1d20/train')
test_path= ('own_dataset/1d20/test')

In [4]:
batch_size_train=75  #,batch_size= batch_size_train
batch_size_valid=75
targetsize= 80
datagen=ImageDataGenerator(rotation_range=180,height_shift_range=0.1,rescale=1.0/255.0
                           ,zoom_range=[0.7,1.1],brightness_range=[0.4,1.2])
train_batches= datagen.flow_from_directory(train_path, target_size=(targetsize,targetsize), shuffle=True,
                                                                      classes=os.listdir('own_dataset/1d20/train'),
                                                                      batch_size= batch_size_train)
test_batches= datagen.flow_from_directory(test_path,  target_size=(targetsize,targetsize), shuffle=True,
                                                                     classes=os.listdir('own_dataset/1d20/train'),
                                                                     batch_size= batch_size_train)

Found 14197 images belonging to 20 classes.
Found 2881 images belonging to 20 classes.


In [5]:
def plots(ims, figsize=(20,10), rows=1, interp= False, titles= None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims= ims.transpose((0,1,2,3))
    f= plt.figure(figsize=figsize)
    cols= len(ims)//rows if len(ims) %2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=12)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [6]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten

from tensorflow.keras.models import model_from_json


In [7]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 5e-4 *0.99**x)

In [8]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(targetsize,targetsize, 3)) )
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='SAME', activation='relu'))
model.add(MaxPooling2D((2, 2)))


model.add(Conv2D(filters=128, kernel_size=(3,3), padding='SAME', activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding='SAME', activation='relu'))
model.add(MaxPooling2D((2, 2)))

#model.add(Conv2D(filters=512, kernel_size=(3,3), padding='SAME', activation='relu'))
#model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(20, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 80, 80, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 256)       2

In [ ]:
model.compile(Adam(lr=.0002), loss='categorical_crossentropy', metrics= ['accuracy'])
history2= model.fit_generator(train_batches, steps_per_epoch= len(train_batches) , 
                             validation_data=test_batches, validation_steps= len(test_batches), 
                             epochs=225)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/225
190/190 [==============================] - 363s 2s/step - loss: 2.9868 - accuracy: 0.0639 - val_loss: 2.9805 - val_accuracy: 0.0684
Epoch 2/225
190/190 [==============================] - 351s 2s/step - loss: 2.9428 - accuracy: 0.0775 - val_loss: 2.9099 - val_accuracy: 0.0889
Epoch 3/225
190/190 [==============================] - 350s 2s/step - loss: 2.8488 - accuracy: 0.1092 - val_loss: 2.7883 - val_accuracy: 0.1274
Epoch 4/225
190/190 [==============================] - 352s 2s/step - loss: 2.7170 - accuracy: 0.1361 - val_loss: 2.6756 - val_accuracy: 0.1434
Epoch 5/225
190/190 [==============================] - 357s 2s/step - loss: 2.5966 - accuracy: 0.1663 - val_loss: 2.5974 - val_accuracy: 0.1520
Epoch 6/225
190/190 [==============================] - 352s 2s/step - loss: 2.5276 - accuracy: 0.1786 - val_loss: 2.5114 - val_accuracy: 0.1826
Epoch 7/225
190/190 [==============================] - 355s 

Epoch 56/225
190/190 [==============================] - 220s 1s/step - loss: 1.5130 - accuracy: 0.4847 - val_loss: 1.5591 - val_accuracy: 0.4728
Epoch 57/225
190/190 [==============================] - 220s 1s/step - loss: 1.4918 - accuracy: 0.4914 - val_loss: 1.5063 - val_accuracy: 0.4946
Epoch 58/225
190/190 [==============================] - 220s 1s/step - loss: 1.4830 - accuracy: 0.5006 - val_loss: 1.5928 - val_accuracy: 0.4502
Epoch 59/225
190/190 [==============================] - 221s 1s/step - loss: 1.4877 - accuracy: 0.4959 - val_loss: 1.5499 - val_accuracy: 0.4648
Epoch 60/225
190/190 [==============================] - 220s 1s/step - loss: 1.4695 - accuracy: 0.4995 - val_loss: 1.4791 - val_accuracy: 0.5089
Epoch 61/225
190/190 [==============================] - 220s 1s/step - loss: 1.4521 - accuracy: 0.5047 - val_loss: 1.4855 - val_accuracy: 0.4866
Epoch 62/225
190/190 [==============================] - 220s 1s/step - loss: 1.4545 - accuracy: 0.5050 - val_loss: 1.4400 - val_ac

190/190 [==============================] - 226s 1s/step - loss: 1.1595 - accuracy: 0.6053 - val_loss: 1.2025 - val_accuracy: 0.5988
Epoch 113/225
190/190 [==============================] - 217s 1s/step - loss: 1.1358 - accuracy: 0.6133 - val_loss: 1.1165 - val_accuracy: 0.6237
Epoch 114/225
190/190 [==============================] - 220s 1s/step - loss: 1.1249 - accuracy: 0.6206 - val_loss: 1.1056 - val_accuracy: 0.6251
Epoch 115/225
190/190 [==============================] - 217s 1s/step - loss: 1.1362 - accuracy: 0.6205 - val_loss: 1.1247 - val_accuracy: 0.6067
Epoch 116/225
190/190 [==============================] - 217s 1s/step - loss: 1.1481 - accuracy: 0.6159 - val_loss: 1.1886 - val_accuracy: 0.6036
Epoch 117/225
190/190 [==============================] - 216s 1s/step - loss: 1.1447 - accuracy: 0.6146 - val_loss: 1.0818 - val_accuracy: 0.6425
Epoch 118/225
190/190 [==============================] - 216s 1s/step - loss: 1.1308 - accuracy: 0.6188 - val_loss: 1.1987 - val_accuracy:

In [ ]:
model_json = model.to_json()
with open("model_1d20.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/

model.save('model_1d20.h5')